# 與 CSV 檔聊天

In [ ]:
!pip install openai
!pip install tiktoken
!pip install langchain
!pip install chromadb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 7.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 858.2/858.2 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = 'sk-'

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

Mounted at /content/gdrive


## 讀取 csv 檔案 - 使用 LangChain 封裝 class 作法

參考連結：[LangChain-csv loader](https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/csv.html)

In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader

In [ ]:
loader = CSVLoader(file_path='/content/gdrive/My Drive/LLM/pokemon.csv')

In [ ]:
print(loader)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [ ]:
data = loader.load_and_split(splitter)

In [ ]:
data[0]

Document(page_content='#: 1\nName: Bulbasaur\nType 1: Grass\nType 2: Poison\nTotal: 318\nHP: 45\nAttack: 49\nDefense: 49\nSp. Atk: 65\nSp. Def: 65\nSpeed: 45\nGeneration: 1\nLegendary: False', metadata={'source': '/content/gdrive/My Drive/LLM/pokemon.csv', 'row': 0})

### 讀取 Embeddings

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings

In [ ]:
embeddings = OpenAIEmbeddings()

## 存放至向量資料庫 - Chroma

In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
db = Chroma.from_documents(data, embeddings)

In [ ]:
db

## 建構 RetrievalQA Chain

In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

In [ ]:
retriever = db.as_retriever()

In [ ]:
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever, input_key="question")

In [ ]:
query = "What is Pikachu stats?"
response = qa({"question": query})
print(response['question'])
print(response['result'])

What is Pikachu stats?
 Pikachu's stats are HP: 35, Attack: 55, Defense: 40, Sp. Atk: 50, Sp. Def: 50, Speed: 90.


In [ ]:
query = "What are the columns in the document?"
response = qa({"question": query})
print(response['question'])
print(response['result'])

What are the columns in the document?
 The columns in the document are Name, Type 1, Type 2, Total, HP, Attack, Defense, Sp. Atk, Sp. Def, Speed, Generation, and Legendary.


In [ ]:
query = "What is the most powerful Pokemon in terms of HP? And Who own it?"
response = qa({"question": query})
print(response['question'])
print(response['result'])

What is the most powerful Pokemon in terms of HP? And Who own it?
 Mewtwo has the highest HP out of these four Pokemon, with 106 HP. It is owned by Generation 1 and is a Legendary Pokemon.


## 使用 Agent

In [ ]:
from langchain.agents import create_csv_agent
from langchain.llms import OpenAI

In [ ]:
agent = create_csv_agent(OpenAI(temperature=0), '/content/gdrive/My Drive/LLM/pokemon.csv', verbose=True)

In [ ]:
agent.run("What is Pikachu stats?")



> Entering new AgentExecutor chain...
Thought: I need to find Pikachu in the dataframe
Action: python_repl_ast
Action Input: df[df['Name'] == 'Pikachu']
Observation:      #     Name    Type 1 Type 2  Total  HP  Attack  Defense  Sp. Atk  \
30  25  Pikachu  Electric    NaN    320  35      55       40       50   

    Sp. Def  Speed  Generation  Legendary  
30       50     90           1      False  
Thought: I now know the final answer
Final Answer: Pikachu has a total stat of 320, HP of 35, Attack of 55, Defense of 40, Sp. Atk of 50, Sp. Def of 50, and Speed of 90.

> Finished chain.


'Pikachu has a total stat of 320, HP of 35, Attack of 55, Defense of 40, Sp. Atk of 50, Sp. Def of 50, and Speed of 90.'

In [ ]:
agent.run("how many rows are there?")



> Entering new AgentExecutor chain...
Thought: I need to count the rows
Action: python_repl_ast
Action Input: len(df)
Observation: 800
Thought: I now know the final answer
Final Answer: 800 rows

> Finished chain.


'800 rows'

In [ ]:
agent.run("What is the most powerful Pokemon in terms of HP? And Who own it?")



> Entering new AgentExecutor chain...
Thought: I need to find the Pokemon with the highest HP value.
Action: python_repl_ast
Action Input: df[df['HP'] == df['HP'].max()]
Observation:        #     Name  Type 1 Type 2  Total   HP  Attack  Defense  Sp. Atk  \
261  242  Blissey  Normal    NaN    540  255      10       10       75   

     Sp. Def  Speed  Generation  Legendary  
261      135     55           2      False  
Thought: I now know the final answer
Final Answer: The most powerful Pokemon in terms of HP is Blissey, with 255 HP.

> Finished chain.


'The most powerful Pokemon in terms of HP is Blissey, with 255 HP.'